In [5]:
import os

from mistralai.client import MistralClient
from mistralai.models.jobs import WandbIntegrationIn, TrainingParameters
from dotenv import load_dotenv

load_dotenv()

mistral_api_key = os.environ["MISTRAL_API_KEY"]
wandb_api_key = os.environ["WANDB_API_KEY"]

client = MistralClient(api_key=mistral_api_key)

train_file = "chipstral_train.jsonl"
eval_file = "chipstral_eval.jsonl"

In [3]:
with open(train_file, "rb") as f:
    chipstral_train = client.files.create(file=(train_file, f))
    
chipstral_train

FileObject(id='e22e42c4-ea4c-488b-ae7e-7ffb1f64de17', object='file', bytes=45287873, created_at=1719764144, filename='chipstral_train.jsonl', purpose='fine-tune')

In [4]:
with open(eval_file, "rb") as f:
    chipstral_eval = client.files.create(file=(eval_file, f))
    
chipstral_eval

FileObject(id='b501c69c-69d8-4e4e-bb86-1a669186169a', object='file', bytes=1886057, created_at=1719764145, filename='chipstral_eval.jsonl', purpose='fine-tune')

In [8]:
created_jobs = client.jobs.create(
    model="open-mistral-7b",
    training_files=[chipstral_train.id],
    validation_files=[chipstral_eval.id],
    #dry_run=True,
    hyperparameters=TrainingParameters(
        training_steps=100,
        learning_rate=0.0001,
    ),
    suffix="chipstral-model",
    integrations=[
        WandbIntegrationIn(
            project="mistral7b_finetuning",
            run_name="run_chipstral",
            api_key=wandb_api_key,
        ).dict()
    ]
)

created_jobs


Job(id='e84d1b84-2a82-454d-8a50-b70faa3ac6f0', hyperparameters=TrainingParameters(training_steps=100, learning_rate=0.0001), fine_tuned_model=None, model='open-mistral-7b', status='QUEUED', job_type='FT', created_at=1719764338, modified_at=1719764338, training_files=['e22e42c4-ea4c-488b-ae7e-7ffb1f64de17'], validation_files=['b501c69c-69d8-4e4e-bb86-1a669186169a'], object='job', integrations=[WandbIntegration(type='wandb', project='mistral7b_finetuning', name=None, run_name='run_chipstral')])

In [12]:
retrieved_jobs = client.jobs.retrieve(created_jobs.id)
retrieved_jobs

DetailedJob(id='e84d1b84-2a82-454d-8a50-b70faa3ac6f0', hyperparameters=TrainingParameters(training_steps=100, learning_rate=0.0001), fine_tuned_model='ft:open-mistral-7b:9db6a39e:20240630:chipstral-model:e84d1b84', model='open-mistral-7b', status='SUCCESS', job_type='FT', created_at=1719764338, modified_at=1719764960, training_files=['e22e42c4-ea4c-488b-ae7e-7ffb1f64de17'], validation_files=['b501c69c-69d8-4e4e-bb86-1a669186169a'], object='job', integrations=[WandbIntegration(type='wandb', project='mistral7b_finetuning', name=None, run_name='run_chipstral')], events=[Event(name='status-updated', data={'status': 'SUCCESS'}, created_at=1719764960), Event(name='status-updated', data={'status': 'RUNNING'}, created_at=1719764338), Event(name='status-updated', data={'status': 'QUEUED'}, created_at=1719764338)], checkpoints=[Checkpoint(metrics=Metric(train_loss=0.003845, valid_loss=None, valid_mean_token_accuracy=None), step_number=100, created_at=1719764917), Checkpoint(metrics=Metric(train_